# Lane Boundary Segmentation

## Setting up Colab

You can delete this "Setting up Colab" section if you work locally and do not want to use Google Colab

In [ ]:
colab_nb = 'google.colab' in str(get_ipython())

In [ ]:
if colab_nb:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
if colab_nb:
  %cd drive/My\ Drive/aad/code/solutions/lane_detection

In [ ]:
if colab_nb:
    !pip install fastseg    !pip install fastai --upgrade


## 1. Loading data

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import cv2
import matplotlib.pyplot as plt
import re
import sys
sys.path.append("../../util")

If you have collected data yourself in a folder "data" using `collect_data.py` and you want to use it for training, set the boolean in the next cell to `True`

In [ ]:
own_data = False

In [ ]:
if own_data:
    from seg_data_util import sort_collected_data
    # copy and sort content of 'data' into 'data_lane_segmentation' folder:
    sort_collected_data()
    # Since data was copied, you can remove files in 'data' directory afterwards
else:
    # if you stopped the download before completion, please delete the 'data_lane_segmentation' folder and run this cell again
    from seg_data_util import download_segmentation_data
    download_segmentation_data()

Independent of what you chose, you will have a directory 'data_lane_segmentation' now

In [ ]:
from fastai.vision.all import *

In [ ]:
DATA_DIR = "data_lane_segmentation"


x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'train_label')

x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'val_label')

## 2. Import fastai

In [ ]:
from fastai.vision.all import *

In [ ]:
# some other usefuls libs
import os
import matplotlib.pyplot as plt
import cv2
def get_image_array_from_fn(fn):
    image = cv2.imread(fn)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

## 3. Prepare data for usage with fastai library

We will use a modified version of the fastai code for image segmentation that is given in the fastai documentation: https://docs.fast.ai/tutorial.vision.html#Segmentation---With-the-data-block-API

### 3.1 label_func

In [ ]:
from sys import platform
folder_token = "\\" if platform == "win32" else "/"

In [ ]:
# function that takes filename of a training image 'fn' and returns the filename of the corresponding label image
def label_func(fn): 
    return str(fn).replace(".png", "_label.png").replace("train", "train_label").replace("val"+folder_token, "val_label"+folder_token)

In [ ]:
# pick the first image from the training directory and show it
sample_fn = os.path.join(x_valid_dir, os.listdir(x_valid_dir)[0])
print(sample_fn)
plt.imshow(get_image_array_from_fn(sample_fn));

In [ ]:
# get corresponding label image using our 'label_func' function
label_fn = label_func(sample_fn)
print(label_fn)
# we multiply the image intensity by 100 to make lane lines visible for the human eye:
plt.imshow(100*get_image_array_from_fn(label_fn)); 

### 3.2 get_image_files

For the datablock API of the fastai library we need a function that takes a file path and returns a list of all the training images. We cannot **directly** use the built-in function 'get_image_files', since it would fetch all images, even the label images. Hence we define a function 'my_get_image_files' that does the same thing as 'get_image_files', just that it only looks into the folders "train" and "val". It will not look into "train_label" and "val_label". We can do this by inspecting the documentation of get_image_files on [docs.fast.ai](https://docs.fast.ai/data.transforms.html#get_image_files) and using ['partial'](https://www.geeksforgeeks.org/partial-functions-python/)

In [ ]:
my_get_image_files = partial(get_image_files, folders=["train", "val"])

### 3.3 DataBlock, DataLoaders and data augmentation

In [ ]:
codes = np.array(['back', 'left','right'],dtype=str)

In [ ]:
carla = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items = my_get_image_files,
                   get_y = label_func,
                   splitter = FuncSplitter(lambda x: str(x).find('validation_set')!=-1),
                   batch_tfms=aug_transforms(do_flip=False, p_affine=0, p_lighting=0.75))

In [ ]:
dls = carla.dataloaders(Path(DATA_DIR), path=Path("."), bs=2)

In [ ]:
dls.show_batch(max_n=6)

## 4. Model and training

In [ ]:
from fastseg import MobileV3Small

model = MobileV3Small(num_classes=3, use_aspp=True, num_filters=64)

In [ ]:
learn = Learner(dls, model, metrics=[DiceMulti(), foreground_acc])

In [ ]:
learn.fine_tune(5)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))

In [ ]:
torch.save(learn.model, './fastai_model.pth')

# Experiments with inference

In [ ]:
import cv2
img = cv2.imread(str(get_image_files(x_valid_dir)[3]))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(np.array(learn.predict(img)[0]))

In [ ]:
# %timeit learn.predict(img); # => more than 100ms!

In [ ]:
def get_pred_for_mobilenet(model, img_array):
    with torch.no_grad():
        image_tensor = img_array.transpose(2,0,1).astype('float32')/255
        x_tensor = torch.from_numpy(image_tensor).to("cuda").unsqueeze(0)
        model_output = F.softmax( model.forward(x_tensor), dim=1 ).cpu().numpy()
    return model_output

In [ ]:
learn.model.eval();

In [ ]:
plt.imshow(get_pred_for_mobilenet(learn.model,img)[0][2])

In [ ]:
%timeit get_pred_for_mobilenet(learn.model,img)

In [ ]:
# this is much faster!!!